<a href="https://colab.research.google.com/github/adrian-ja-projects/train-prediction-project/blob/fea_data_analisys/pl_extraction_api_to_staging.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import datetime
import requests
import json
from os.path import exists

## Extract from API to Staging

In [ ]:
def _get_date_list(num_days: int, start_year: int, start_month: int, start_day: int)-> list:
  """
  method to get the list of json files to download based on the date
  """
  base = datetime.datetime(start_year, start_month, start_day)
  return  [(base + datetime.timedelta(days=x)).strftime("%Y-%m-%d") for x in range(num_days)]

def _get_train_metadata_json(departure_date: str, train_number: str) -> None:
  """
  read data from the api of only one departure date at the time
  """
  url = f"https://rata.digitraffic.fi/api/v1/trains/{departure_date}/{train_number}?include_deleted=false&version=0"
  response = requests.get(url).json()
  return response

def _check_object_exists(file_path: str)-> bool:
  """
  method to check if object exists
  """
  return exists(file_path)

def _overwrite_file(file_path: str)-> None:
  """
  delete a file to overwrite
  """
  delete_file = !rm "$file_path"
  delete_file
  create_file = !touch "$file_path"
  create_file

def _create_file(staging_path: str, file_path: str)-> None:
  """
  create a empty file to write into
  """
  create_dir = !mkdir -p "$staging_path"
  create_dir
  create_empty_file = !touch "$file_path"
  create_empty_file

def write_json_to_staging(departure_date: str, file_format: str, train_number: str, overwrite: bool)-> None:
  """
  call the api and storage the json file into the staging folder with the following path convnetion:
  stating/table_name/departure_month/departure_date/table_name.(format)
  """
  staging_path = f"/content/staging/digitraffic/stg_{train_number}_schedule/{departure_date[0:7]}/{departure_date}/"
  file_name = f"stg_{train_number}_schedule.{file_format}"
  file_path = staging_path+file_name
  if _check_object_exists(file_path):
    if overwrite:
      _overwrite_file(file_path)
  else:
     _create_file(staging_path, file_path)
  with open(file_path, 'w') as json_file:
    data = _get_train_metadata_json(departure_date, train_number)
    json.dump(data, json_file)
    json_file.close()

if __name__ == "__main__":
  #TO-DO-> get this variables from a scheduled pipeline metadata
  num_days = 100
  start_year = 2020
  start_month = 1
  start_day = 1
  print("INFO: Creating list for extraction loop...")
  dates_of_extraction = _get_date_list(num_days, start_year, start_month, start_day)
  print(f"INFO: Extraction dates are between {min(dates_of_extraction)} and {max(dates_of_extraction)}")
  print("INFO: Starting extraction...")
  
  for d in dates_of_extraction:
    #Make an API call per date and write into staging
    #TO-DO-> get this variables from a scheduled pipeline metadata
    departure_date = d
    file_format = "json"
    train_number = "27"
    overwrite = True
    write_json_to_staging(departure_date, file_format, train_number, overwrite)
  print(f"INFO: Extraction completed a total of {len(dates_of_extraction)} file were successfully extracted into the staging area")